In [ ]:
# Importing dependencies
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Loading Data
transform =transforms.Compose([transforms.ToTensor()])

train_dataset=datasets.MNIST('data',train=True,download=True,transform=transform)
test_dataset=datasets.MNIST('data',train=False,download=True,transform=transform)
#transforms.ToTensor() - changing the images into a format - tensors 

In [ ]:
#print(train_dataset[0])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# Define the image classifier model
# class SimpleModel(nn.Module):
#     def __init__(self):
#         super(SimpleModel, self).__init__()
        
#        
#         self.fc1 = nn.Linear(28*28, 128)  # Flatten the 28x28 pixels into 1 row of 784 numbers
#         self.fc2 = nn.Linear(128, 10)     # Output 10 guesses (0-9 numbers)

#     def forward(self, x):
#         x = x.view(-1, 28*28)  # Flatten the image from 28x28 to 1 long line
#         x = torch.relu(self.fc1(x))  # Look at the first part (hidden layer)
#         x = self.fc2(x)  # Guess which number it is (final output)
#         return x

# Define the image classifier model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU()
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 22 * 22, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [ ]:


# build model
model = SimpleModel()

# Define the optimizer and loss function

criterion = nn.CrossEntropyLoss()


optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
epochs = 5
for epoch in range(epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()  # Clear old mistakes
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 0.376335084438324
Epoch 2, Loss: 0.2197747677564621
Epoch 3, Loss: 0.0669187530875206
Epoch 4, Loss: 0.03532875329256058
Epoch 5, Loss: 0.11866281181573868


In [ ]:

correct = 0
total = 0
with torch.no_grad():  # Don't do any training during testing
    for images, labels in test_loader:
        outputs = model(images)  
        _, predicted = torch.max(outputs, 1)  # The best guess
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')  #  how often the robot is it correct


Accuracy: 97.87%


In [17]:
# Save the trained model to a file
torch.save(model.state_dict(), 'model.pth')
